# Distributive Access To Opportunities in Calgary, Canada

Welcome! This workbook is the first of three notebooks that are intended to be worked through in sequence. In this notebook we will introduce the data analysis approach and concepts of access to opportunities in a broader sense. We'll then explore the demographic and opportunity data that we will use to calcualte access to opportunities in future workbooks.

## A Recipe for Quantiative Access-Based Equity Analysis

### Data Inputs

The image below shows how this project's data sources and workflow is structured. This follows a relatively standard way of performing these types of analyses:

![title](img/data_flow.png)

There are four key sources of data that we use for this type of analysis:

1. **Transit Schedules** in the form of General Transit Feed Specification (GTFS) data. These are produced by transit agencies and often published on transit agency websites. Calgary Transit's GTFS schedule published on March 3, 2023 is provided in this repository.
2. **A Street Map network** in the form of an OpenStreetMap (OSM) network file. These can be downloaded from sources such as [GeoFabrik](https://download.geofabrik.de/) or from a custom boudning box from services like BBBike [BBBike](https://extract.bbbike.org/). Calgary's OSM extract (a PBF file) is provided in this repository.
3. **Opportunity Locations** or *destinations* are places where people would like to visit. These can take on any number of forms, and can be more abstract (jobs) or specific (pharmacies). These opportunity locations either need to be in specific point form (lat/lon coordinates), or counts of them should be encoded in whatever zone format you are using for the analysis area. In this project we are using hospitals and daycare locations.
4. **Demographic Data** or population data for the area of study. These are usually encoded in the zone format you are using as an underlying analysis zone and contain counts of individuals from various demographic groups. In this case, we are studying total population, visible minorities, low-income individuals, and single parent households.

Since we can't feasibly measure access from every single point in space in a given study area (Calgary), we need some sort of zone system. In this example we'll be using the *dissemination area* zones defined by the Canadian Census, which are sized to contain similar numbers of residents.

### Computation
We will need to run a few computations in order to get some useful final results. In particular, we will need to calculate the following:
1. **Transit travel times** between the centres (centroids) of our zone systems and our opportunity locations (daycares and hospitals). How we do this is covered in more detail in workbook `2 - Travel Times.ipynb`.
2. **Shortest travel time and cumulatve access** are two general ways to calculate access metrics from a given zone. We will cover those in more detail in Workbook `3 - Access and Equity.ipynb`.
3. **Population weighted summaries by demographic groups** of our access metrics lets us compare differences in the systematic or average benefit conveyed by transit across populations. We can then learn - for example - if visible minorities have an average longer trip to reach a hospital compared with the average population. Calculating, visualizing, and interpreting these results is also discussed in `3 - Access and Equity.ipynb`

## Data Exploration
Before we dive into the calculation, let's take a moment to explore the data that we are going to use for our project. In particular let's explore the opportunity locations and demographics.

### Imports and Data Setup
Let's start by importing the relevant packages we need for these visuals: Pandas/Geopandas for tabular/spatial analysis, and Altair for creating charts and maps. We'll also make our life easy here and set a master projection which we can reference for maps later on. Feel free to play with projections, your options in this case can be found [in the Vega documentation](https://vega.github.io/vega/docs/projections/).

We will also read in a form of basemap using the dissemination areas data we have, so we can provide some context to the map for ourselves.

In [ ]:
import altair as alt
import geopandas as gpd
import pandas as pd
PROJECTION = "mercator"

da = gpd.read_file("data/da_with_locations.geojson")
da.head()

## Opportunity Locations
To make sure we understand roughly what our dataset looks like and how we can validate it, let's start by taking a visual look at opportunity locations. In particular, we want to know roughly where our two different opportunity sets are. The workflow here is pretty straightforward:

1. Read in the spatial location dataset
2. Generate a map plot of those location points

### Hospital Locations

In [ ]:
hospitals = gpd.read_file("data/hospital_locations.geojson")

basemap = alt.Chart(da).mark_geoshape(fill="lightgrey", stroke="lightgrey")

hosp = alt.Chart(hospitals).mark_circle(size=150).encode(
    latitude='geometry.coordinates[1]:Q',
    longitude='geometry.coordinates[0]:Q',
    tooltip='facility_name:N'
).project(
    PROJECTION
).properties(
    width=500,
    height=400
)

basemap + hosp

### Childcare Locations
Childcare locations are more abundant, and they also have a value attached to them in terms of the number of seats. Let's size the dots on the map based on the number of seats so we can see if there are clusters both in terms of number of options but also in terms of scale.

In [ ]:
daycares = gpd.read_file("data/daycare_locations.geojson")

basemap = alt.Chart(da).mark_geoshape(fill="lightgrey", stroke="lightgrey")

dayc = alt.Chart(daycares).mark_circle().encode(
    latitude='latitude:Q',
    longitude='longitude:Q',
    size=alt.Size("capacity:Q"),
    tooltip='name:N'
).project(
    "mercator"
).properties(
    width=500,
    height=400
)

basemap + dayc

## Demographics
Let's visualize our demographic categories on a choropleth map to see if we can identify some larger spatial trends. The flow here is:

1. Read our demographic data
2. Join our demographic data to the dissemination areas
3. Calculate fractional concetrations of demographics
4. Plot these demographics on a choropleth map.

Let's start by reading in our demographic data and joining it to the dissemination areas:

In [ ]:
demo = pd.read_csv("data/demographics.csv")
# We need to ensure that we cast this data as an integer so we can join our datasets together
da["dauid"] = da["dauid"].astype(int)
da_demo = pd.merge(da, demo, on="dauid")
da_demo.head()

### Visible Minority Populations
Let's start by looking at the relative concentration of visible minority populations:

In [ ]:
da_demo['vismin_pct'] = 100*da_demo["vismin_vismin"]/da_demo["vismin_total"]

alt.Chart(da_demo).mark_geoshape().encode(
    color=alt.Color('vismin_pct:Q', title="% Visible Minority")
).properties(
    width=800,
    height=1000
).project(
    PROJECTION
)

### Single Mother Households

In [ ]:
da_demo['single_mother_pct'] = 100*da_demo["fam_onemother"]/da_demo["fam_total"]

alt.Chart(da_demo).mark_geoshape().encode(
    color=alt.Color('single_mother_pct:Q', title="% Singe Mother")
).properties(
    width=800,
    height=1000
).project(
    PROJECTION
)

### Low Income (LICO)

In [ ]:
da_demo['lico_pct'] = 100*da_demo["lico_lico"]/da_demo["lico_total"]

alt.Chart(da_demo).mark_geoshape().encode(
    color=alt.Color('lico_pct:Q', title="% Low-Income (LICO)", scale=alt.Scale(scheme="oranges"))
).properties(
    width=800,
    height=1000
).project(
    PROJECTION
)

well that's strange, there's one DA that's causing particular trouble...

In [ ]:
da_demo.sort_values("lico_pct", ascending=False).head()

so let's plot it again but remove really low numbers.

In [ ]:
da_demo['lico_pct'] = 100*da_demo["lico_lico"]/da_demo["lico_total"]

alt.Chart(da_demo[da_demo["lico_total"] > 10]).mark_geoshape().encode(
    color=alt.Color('lico_pct:Q', title="% Low-Income (LICO)", scale=alt.Scale(scheme="oranges"))
).properties(
    width=800,
    height=1000
).project(
    PROJECTION
)